# Código para unir tabelas com o mesmo formato e realizar a limpeza.

Os arquivos utilizados nessa etapa são o resultado da etapa anterior, consistindo em 15 arquivos .xlsx contemplando dados de 2014 a 2023. Serão gerados a partir daqui quatro diferentes datasets contemplando os períodos de:

1) 2014 a novembro de 2018

2) Dezembro de 2018 até final 2019

3) Primeiro semestre de 2020

4) Segundo semestre de 2020 até final de 2023
 

In [1]:
import pandas as pd
import sys
from unidecode import unidecode

def MostraPorcentagem(df, coluna):
    contagem = df[coluna].value_counts()
    
    total = df.shape[0] 
    porcentagem = (contagem / total) * 100
    
    resultado = pd.DataFrame({
        'Contagem': contagem,
        'Porcentagem': porcentagem
    })

    resultado['Porcentagem'] = resultado['Porcentagem'].map('{:.2f}%'.format)
    
    print(resultado)

def MostraValoresUnicos(df, coluna):
    valoresUnicos = df[coluna].dropna().unique()
    valoresUnicos = [str(valor) for valor in valoresUnicos]

    valoresOrdenados = sorted(valoresUnicos)

    for valor in valoresOrdenados:
        print(valor)

def MostraValoresUnicosExceto(df, colunasExcluidas):
    for coluna in df.columns:
        if coluna not in colunasExcluidas:
            print(f"/////////////////////////////////////////////////{coluna}")
            MostraValoresUnicos(df, coluna)

def ClassificaProfissao(profissao, arquivoComProfissoes):
    if profissao == "ESTUDANTE":
        return "ESTUDANTE"
    elif profissao in arquivoComProfissoes['Grupo 1'].values:
        return "GRUPO 1"
    elif profissao in arquivoComProfissoes['Grupo 2'].values:
        return "GRUPO 2"
    elif profissao in arquivoComProfissoes['Grupo 3'].values:
        return "GRUPO 3"
    elif profissao in arquivoComProfissoes['Grupo 4'].values:
        return "GRUPO 4"
    elif profissao in arquivoComProfissoes['Grupo 5'].values:
        return "GRUPO 5"
    else: 
        return "OUTROS"
    
def PadronizaMunicipio(municipio):
    if isinstance(municipio, str):
        municipioLimpo = municipio.split('|')[-1].strip()
        return municipioLimpo
    return municipio

def PadronizaPais(pais):
    if isinstance(pais, str) and '|' in pais:
        paisLimpo = pais.split('|')[-1].strip()
        return paisLimpo
    return pais

## 1) Tratamento de 2014 a 2018-1

In [2]:
# Le os arquivos de 2014 a 2018-1
tabela2014 = pd.read_excel("2014.xlsx", dtype=str)
tabela2015 = pd.read_excel("2015.xlsx", dtype=str)
tabela2016 = pd.read_excel("2016.xlsx", dtype=str)
tabela2017 = pd.read_excel("2017.xlsx", dtype=str)
tabela2018_1 = pd.read_excel("2018-1.xlsx", dtype=str)

In [3]:
# Lê arquivo com a lista de municipios do Brasil
arquivoComMunicipios = pd.read_csv("municipios.csv", dtype=str, delimiter=';', encoding = "ISO-8859-1")
arquivoComMunicipios['MUNICÍPIO - TOM'] = arquivoComMunicipios['MUNICÍPIO - TOM'].apply(unidecode)

In [4]:
# Unifica as tabelas
tabela2014_2018_1 = pd.concat([tabela2014, tabela2015, tabela2016, tabela2017, tabela2018_1], ignore_index=True)
tabela2014_2018_1.columns

Index(['Data de cadastro', 'Tipo de violência', 'Violência familiar',
       'Denunciante', 'Filhos vítima', 'Dependência Financeira da Vítima',
       'Sexo da vítima', 'Raça\Cor da vítima', 'Faixa etária da vítima',
       'Grau de instrução da vítima', 'Frequência', 'Coabitação contexto',
       'Tempo de violência contexto', 'Risco contexto',
       'Relação vítima-suspeito', 'Sexo do suspeito', 'Raça\Cor do suspeito',
       'Faixa etária do suspeito', 'Grau de instrução do suspeito',
       'Drogas Álcool Suspeito', 'Comportamento efeito agressor',
       'Filhos violência', 'Violações', 'UF', 'Município', 'Residência'],
      dtype='object')

In [5]:
# Limpezas especificas
for coluna in tabela2014_2018_1.columns: tabela2014_2018_1[coluna] = tabela2014_2018_1[coluna].replace(['NAO SOUBE INFORMAR'], '')
tabela2014_2018_1['UF'] = tabela2014_2018_1['UF'].str.strip()
tabela2014_2018_1['UF'] = tabela2014_2018_1['UF'].mask(~tabela2014_2018_1['UF'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

tabela2014_2018_1['Município'] = tabela2014_2018_1['Município'].str.replace(r'\s+\(\w{2}\)$', '', regex=True)
tabela2014_2018_1['Município'] = tabela2014_2018_1['Município'].where(tabela2014_2018_1['Município'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

substituicoes = {
    # Alterados
    "AMIGA": "AMIGO(A)",
    "AMIGO": "AMIGO(A)",

    "CHEFA": "CHEFE(A)",
    "CHEFE": "CHEFE(A)",

    "NAMORADA (LESBICA)":       "NAMORADO(A)/COMPANHEIRO(A)",
    "NAMORADO":                 "NAMORADO(A)/COMPANHEIRO(A)",
    "COMPANHEIRA (LESBICA)":    "NAMORADO(A)/COMPANHEIRO(A)",
    "COMPANHEIRO":              "NAMORADO(A)/COMPANHEIRO(A)",

    "EX NAMORADA (LESBICA)":        "EX-NAMORADO(A)/EX-COMPANHEIRO(A)",
    "EX NAMORADO":                  "EX-NAMORADO(A)/EX-COMPANHEIRO(A)",
    "EX COMPANHEIRA (LESBICA)":     "EX-NAMORADO(A)/EX-COMPANHEIRO(A)",
    "EX COMPANHEIRO":               "EX-NAMORADO(A)/EX-COMPANHEIRO(A)",

    "CONHECIDA": "CONHECIDO(A)",
    "CONHECIDO": "CONHECIDO(A)",

    "CUNHADA": "CUNHADO(A)",
    "CUNHADO": "CUNHADO(A)",

    "DESCONHECIDA": "DESCONHECIDO(A)",
    "DESCONHECIDO": "DESCONHECIDO(A)",

    "EX CONJUGE": "EX-CONJUGE",

    "FILHA/ENTEADA": "FILHO(A)/ENTEADO(A)",
    "FILHO/ENTEADO": "FILHO(A)/ENTEADO(A)",

    "IRMA":  "IRMAO(A)",
    "IRMAO": "IRMAO(A)",

    "PROPRIETARIA DO IMOVEL EM QUE A VITIMA RESIDE": "PROPRIETARIO(A) DO IMOVEL EM QUE A VITIMA RESIDE",
    "PROPRIETARIO DO IMOVEL EM QUE A VITIMA RESIDE": "PROPRIETARIO(A) DO IMOVEL EM QUE A VITIMA RESIDE",
    
    "SOGRA": "SOGRO(A)",
    "SOGRO": "SOGRO(A)",

    "VIZINHA": "VIZINHO(A)",
    "VIZINHO": "VIZINHO(A)",

    # Não alterados
    "CONJUGE": "CONJUGE",
    "COLEGA DE TRABALHO": "COLEGA DE TRABALHO",
    "AMANTE": "AMANTE",
    "MADRASTA": "MADRASTA",
    "MAE": "MAE",
    "NAO SOUBE INFORMAR": "NAO SOUBE INFORMAR",
    "OUTRO(A)": "OUTRO(A)",
    "PADRASTO": "PADRASTO",
    "PAI": "PAI",
    "PARENTES": "PARENTES",
    "PROFISSIONAL DA SAUDE": "PROFISSIONAL DA SAUDE",
    "PROFISSIONAL DA SEGURANCA PUBLICA": "PROFISSIONAL DA SEGURANCA PUBLICA",
}
tabela2014_2018_1['Relação vítima-suspeito'] = tabela2014_2018_1['Relação vítima-suspeito'].replace(substituicoes)

substituicoes = {
    # Alterados  
    "NAMORADA (LESBICA)":      "NAMORADO(A)/COMPANHEIRO(A)",
    "NAMORADO":                "NAMORADO(A)/COMPANHEIRO(A)",
    "COMPANHEIRA (LESBICA)":   "NAMORADO(A)/COMPANHEIRO(A)",
    "COMPANHEIRO":             "NAMORADO(A)/COMPANHEIRO(A)",

    "EX COMPANHEIRA (LESBICA)":     "EX-NAMORADO(A)/EX-COMPANHEIRAO(O)",
    "EX COMPANHEIRO":               "EX-NAMORADO(A)/EX-COMPANHEIRAO(O)",
    "EX NAMORADA (LESBICA)":        "EX-NAMORADO(A)/EX-COMPANHEIRAO(O)",
    "EX NAMORADO":                  "EX-NAMORADO(A)/EX-COMPANHEIRAO(O)",
    
    "IRMA(O)": "IRMAO(A)",

    # Não alterados
    "OUTRO(A)": "OUTRO(A)",
    "PADRASTO": "PADRASTO",
    "PAI": "PAI",
    "PARENTES": "PARENTES",
    "PROFISSIONAL QUE ATENDEU A VITIMA": "PROFISSIONAL QUE ATENDEU A VITIMA",
    "PROPRIETARIO(A) DO IMOVEL EM QUE A VITIMA RESIDE": "PROPRIETARIO(A) DO IMOVEL EM QUE A VITIMA RESIDE",
    "SOGRO(A)": "SOGRO(A)",
    "VITIMA": "VITIMA",
    "VIZINHO(A)": "VIZINHO(A)",
    "MADRASTA": "MADRASTA",
    "MAE": "MAE",
    "FILHO(A)/ENTEADO(A)": "FILHO(A)/ENTEADO(A)",
    "AMANTE": "AMANTE",
    "AMIGO(A)": "AMIGO(A)",
    "COLEGA DE TRABALHO": "COLEGA DE TRABALHO",
    "CONHECIDO(A)": "CONHECIDO(A)",
    "CONJUGE": "CONJUGE",
    "CUNHADO(A)": "CUNHADO(A)",
    "DESCONHECIDO(A)": "DESCONHECIDO(A)"
}
tabela2014_2018_1['Denunciante'] = tabela2014_2018_1['Denunciante'].replace(substituicoes)


In [6]:
# Salva a tabela unificada
tabela2014_2018_1.to_excel("../Etapa 3/2014 a 2018-1.xlsx", index=False)

## 2) Tratamento de 2018-2 a 2019

In [ ]:
# Le os arquivos de 2018-2 a 2019
tabela2018_2 = pd.read_excel("2018-2.xlsx", dtype=str)
tabela2019 = pd.read_excel("2019.xlsx", dtype=str)

In [ ]:
# Lê arquivo com a lista de municipios do Brasil
arquivoComMunicipios = pd.read_csv("municipios.csv", dtype=str, delimiter=';', encoding = "ISO-8859-1")
arquivoComMunicipios['MUNICÍPIO - TOM'] = arquivoComMunicipios['MUNICÍPIO - TOM'].apply(unidecode)

In [ ]:
# Unifica as tabelas
tabela2018_2_2019 = pd.concat([tabela2018_2, tabela2019], ignore_index=True)
tabela2018_2_2019.columns

In [ ]:
# Limpezas especificas
for coluna in tabela2018_2_2019.columns: tabela2018_2_2019[coluna] = tabela2018_2_2019[coluna].replace(['NAO INFORMADO', 'N/D'], '')

tabela2018_2_2019['UF da vítima'] = tabela2018_2_2019['UF da vítima'].str.strip()
tabela2018_2_2019['UF da vítima'] = tabela2018_2_2019['UF da vítima'].mask(~tabela2018_2_2019['UF da vítima'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

tabela2018_2_2019['UF do suspeito'] = tabela2018_2_2019['UF do suspeito'].str.strip()
tabela2018_2_2019['UF do suspeito'] = tabela2018_2_2019['UF do suspeito'].mask(~tabela2018_2_2019['UF do suspeito'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

tabela2018_2_2019['Município da vítima'] = tabela2018_2_2019['Município da vítima'].str.replace(r'\s+\(\w{2}\)$', '', regex=True)
tabela2018_2_2019['Município do suspeito'] = tabela2018_2_2019['Município do suspeito'].str.replace(r'\s+\(\w{2}\)$', '', regex=True)

tabela2018_2_2019['Município da vítima'] = tabela2018_2_2019['Município da vítima'].where(tabela2018_2_2019['Município da vítima'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")
tabela2018_2_2019['Município do suspeito'] = tabela2018_2_2019['Município do suspeito'].where(tabela2018_2_2019['Município do suspeito'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

tabela2018_2_2019['Orientação sexual da vítima'] = tabela2018_2_2019['Orientação sexual da vítima'].combine_first(tabela2018_2_2019['Orientação sexual da vítima 2'])

tabela2018_2_2019['Orientação sexual do suspeito'] = tabela2018_2_2019['Orientação sexual do suspeito'].combine_first(tabela2018_2_2019['Orientação sexual do suspeito 2'])

substituicoes = {
    'AMIGO (A)': 'AMIGO(A)',
    'COMPANHEIRO (A)': 'COMPANHEIRO(A)',
    'CUNHADO (A)': 'CUNHADO(A)',
    'EMPREGADO (A)': 'EMPREGADO(A)',
    'ESPOSA': 'ESPOSO(A)',
    'EX-COMPANHEIRO (A)': 'EX-COMPANHEIRO(A)',
    'EX-ESPOSA': 'EX-ESPOSO(A)',
    'EX-NAMORADA': 'EX-NAMORADO(A)',
    'EX-NAMORADO': 'EX-NAMORADO(A)',
    'FILHO (A)': 'FILHO(A)',
    'IRMAO (A)': 'IRMAO(A)',
    'LIDER RELIGIOSO': 'LIDER RELIGIOSO(A)',
    'MAE': 'PAI/MAE',
    'MARIDO': 'ESPOSO(A)',
    'PADRASTO': 'PADRASTO',
    'PAI': 'PAI/MAE',
    'TIO (A)': 'TIO(A)',
    'VIZINHO (A)': 'VIZINHO(A)'
}
tabela2018_2_2019['Relação vítima-suspeito'] = tabela2018_2_2019['Relação vítima-suspeito'].replace(substituicoes)

substituicoes = {
    # Alterados
    "AMIGO (A)": "AMIGO(A)",
    "AVO": "AVO(O)",
    "CHEFE": "CHEFE(A)",
    "COMPANHEIRO (A)": "COMPANHEIRO(A)",
    "CUNHADO (A)": "CUNHADO(A)",
    "DESCONHECIDO(A)": "DESCONHECIDO(A)",
    "EMPREGADO (A)": "EMPREGADO(A)",
    "ENTEADO(A)": "ENTEADO(A)",
    "ESPOSA": "ESPOSA",
    "EX-COMPANHEIRO (A)": "EX-COMPANHEIRO(A)",
    "EX-ESPOSA": "EX-ESPOSA",
    "EX-NAMORADA": "EX-NAMORADA",
    "EX-NAMORADO": "EX-NAMORADO",
    "FAMILIARES": "FAMILIARES",
    "FILHO (A)": "FILHO(A)",
    "GENRO/NORA": "GENRO/NORA",
    "IRMAO (A)": "IRMAO(A)",
    "LIDER RELIGIOSO": "LIDER RELIGIOSO",
    "MADRASTA": "MADRASTA",
    "MAE": "MAE",
    "MARIDO": "MARIDO",
    "NAMORADO(A)": "NAMORADO(A)",
    "NAO INFORMADO": "NAO INFORMADO",
    "NAO SE APLICA": "NAO SE APLICA",
    "NETO(A)": "NETO(A)",
    "PADRASTO": "PADRASTO",
    "PAI": "PAI",
    "PRIMO(A)": "PRIMO(A)",
    "PROFESSOR(A)": "PROFESSOR(A)",
    "PROPRIA VITIMA": "PROPRIA VITIMA",
    "SOBRINHO(A)": "SOBRINHO(A)",
    "SOGRO(A)": "SOGRO(A)",
    "TIO (A)": "TIO(A)",
    "VIZINHO (A)": "VIZINHO(A)"

    # Não alterados
}
tabela2018_2_2019['Relação Demandante Vítima'] = tabela2018_2_2019['Relação Demandante Vítima'].replace(substituicoes)

substituicoes = {
    # Alterados
    ' ESTABELECIMENTO DE ENSINO': 'ESTABELECIMENTO DE ENSINO',
    'UNIDADE PRISIONAL': 'UNIDADE PRISIONAL', 
    'UNIDADE PRISIONAL - PRESIDIO': 'UNIDADE PRISIONAL' 
}
tabela2018_2_2019['Cenário da violação'] = tabela2018_2_2019['Cenário da violação'].replace(substituicoes)


tabela2018_2_2019 = tabela2018_2_2019.drop('Grupo vulnerável', axis=1)
tabela2018_2_2019 = tabela2018_2_2019.drop('Data cenário', axis=1) 
tabela2018_2_2019 = tabela2018_2_2019.drop('Hora cenário', axis=1) 
tabela2018_2_2019 = tabela2018_2_2019.drop('Orientação sexual da vítima 2', axis=1)
tabela2018_2_2019 = tabela2018_2_2019.drop('Orientação sexual do suspeito 2', axis=1)
tabela2018_2_2019 = tabela2018_2_2019.drop('Vitima cod município', axis=1)
tabela2018_2_2019 = tabela2018_2_2019.drop('Suspeito cod municipio', axis=1) 
tabela2018_2_2019 = tabela2018_2_2019.drop('Suspeito pj cod município', axis=1)
tabela2018_2_2019 = tabela2018_2_2019.drop('Suspeito pj município', axis=1)
tabela2018_2_2019 = tabela2018_2_2019.drop('Suspeito pj UF', axis=1)
tabela2018_2_2019 = tabela2018_2_2019.drop('Suspeito pj qualificação jurídica', axis=1)

In [ ]:
# Salva a tabela unificada
tabela2018_2_2019.to_excel("../Etapa 3/2018-2 a 2019.xlsx", index=False)

## 3) Tratamento de 2020-1 

In [ ]:
# Le o arquivos de 2020-1
tabela2020_1 = pd.read_excel("2020-1.xlsx", dtype=str)

In [ ]:
# Lê arquivo com a lista de municipios do Brasil
arquivoComMunicipios = pd.read_csv("municipios.csv", dtype=str, delimiter=';', encoding = "ISO-8859-1")
arquivoComMunicipios['MUNICÍPIO - TOM'] = arquivoComMunicipios['MUNICÍPIO - TOM'].apply(unidecode)

In [ ]:
# Limpezas especificas
for coluna in tabela2020_1.columns: tabela2020_1[coluna] = tabela2020_1[coluna].replace(['NAO INFORMADO', 'N/D'], '')

substituicoes = {
    # Alterados
    "EX- NAMORADO(A)": "EX-NAMORADO(A)",
    
    "EX-ESPOSA": "EX-ESPOSO(A)",
    "EX-MARIDO": "EX-ESPOSO(A)",

    "LIDER RELIGIOSO": "LIDER RELIGIOSO(A)",
    "MARIDO/ ESPOSA": "ESPOSO(A)",

    "MEMBRO": "",

    "PADRASTO/ MADRASTA": "PADRASTO/MADRASTA",
    "PADRINHO/ MADRINHA": "PADRINHO/MADRINHA",
    "PAI/ MAE": "PAI/MAE",
    
    # Não alterado
    "PRIMO(A)": "PRIMO(A)",
    "PROFESSOR(A)": "PROFESSOR(A)",
    "SEM RELACAO": "SEM RELACAO",
    "SOBRINHO(A)": "SOBRINHO(A)",
    "SOGRO(A)": "SOGRO(A)",
    "TIO(A)": "TIO(A)",
    "TRISAVO(O)": "TRISAVO(O)",
    "VIZINHO(A)": "VIZINHO(A)",
    "AVO(O)": "AVO(O)",
    "BISAVO(O)": "BISAVO(O)",
    "AGENTE PENITENCIARIO": "AGENTE PENITENCIARIO",
    "COMPANHEIRO(A)": "COMPANHEIRO(A)",
    "CUIDADOR(A)": "CUIDADOR(A)",
    "CUNHADO(A)": "CUNHADO(A)",
    "DESCONHECIDO(A)": "DESCONHECIDO(A)",
    "DIRETOR(A)": "DIRETOR(A)",
    "ENTEADO(A)": "ENTEADO(A)",
    "EX-COMPANHEIRO(A)": "EX-COMPANHEIRO(A)",
    "FILHO(A)": "FILHO(A)",
    "FUNCIONARIO(A)": "FUNCIONARIO(A)",
    "HIERARQUICAMENTE SUPERIOR": "HIERARQUICAMENTE SUPERIOR",
    "IRMAO(A)": "IRMAO(A)",
    "MESMO NIVEL HIERARQUICO": "MESMO NIVEL HIERARQUICO",
    "N/D": "N/D",
    "NAMORADO(A)": "NAMORADO(A)",
    "NETO(A)": "NETO(A)",
}
tabela2020_1['Relação Demandante Vítima'] = tabela2020_1['Relação Demandante Vítima'].replace(substituicoes)


substituicoes = {
    # Alterados
    "EX- NAMORADO(A)": "EX-NAMORADO(A)",

    "EX-ESPOSA": "EX-ESPOSO(A)",
    "EX-MARIDO": "EX-ESPOSO(A)",

    "LIDER RELIGIOSO": "LIDER RELIGIOSO(A)",

    "MARIDO/ ESPOSA": "MARIDO/ESPOSA",

    "MEMBRO": "",

    "PADRASTO/ MADRASTA": "PADRASTO/MADRASTA",
    "PADRINHO/ MADRINHA": "PADRINHO/MADRINHA",
    "PAI/ MAE": "PAI/MAE",

    # Não alterados
    "PRIMO(A)": "PRIMO(A)",
    "PROFESSOR(A)": "PROFESSOR(A)",
    "SEM RELACAO": "SEM RELACAO",
    "SOBRINHO(A)": "SOBRINHO(A)",
    "SOGRO(A)": "SOGRO(A)",
    "TIO(A)": "TIO(A)",
    "TRISAVO(O)": "TRISAVO(O)",
    "VIZINHO(A)": "VIZINHO(A)",
    "MESMO NIVEL HIERARQUICO": "MESMO NIVEL HIERARQUICO",
    "N/D": "N/D",
    "NAMORADO(A)": "NAMORADO(A)",
    "NETO(A)": "NETO(A)",
    "FILHO(A)": "FILHO(A)",
    "FUNCIONARIO(A)": "FUNCIONARIO(A)",
    "HIERARQUICAMENTE INFERIOR": "HIERARQUICAMENTE INFERIOR",
    "HIERARQUICAMENTE SUPERIOR": "HIERARQUICAMENTE SUPERIOR",
    "INTERNO(A)": "INTERNO(A)",
    "IRMAO(A)": "IRMAO(A)",
    "EX-COMPANHEIRO(A)": "EX-COMPANHEIRO(A)",
    "AGENTE PENITENCIARIO": "AGENTE PENITENCIARIO",
    "AVO(O)": "AVO(O)",
    "BISAVO(O)": "BISAVO(O)",
    "COMPANHEIRO(A)": "COMPANHEIRO(A)",
    "CUIDADOR(A)": "CUIDADOR(A)",
    "CUNHADO(A)": "CUNHADO(A)",
    "DESCONHECIDO(A)": "DESCONHECIDO(A)",
    "DIREITOR(A) DA UNIDADE PRISIONAL": "DIRETOR(A) DA UNIDADE PRISIONAL",
    "DIRETOR(A)": "DIRETOR(A)",
    "ENTEADO(A)": "ENTEADO(A)",
}
tabela2020_1['Relação vítima-suspeito'] = tabela2020_1['Relação vítima-suspeito'].replace(substituicoes)

tabela2020_1['Deficiência da vítima'] = tabela2020_1['Deficiência da vítima'].replace(['NAO'], 'NAO TEM DEFICIENCIA')
tabela2020_1['Deficiência do suspeito'] = tabela2020_1['Deficiência do suspeito'].replace(['NAO'], 'NAO TEM DEFICIENCIA')

tabela2020_1['Sexo da vítima'] = tabela2020_1['Sexo da vítima'].replace(['LGBT'], '')

tabela2020_1 = tabela2020_1.drop('hash', axis=1)
tabela2020_1 = tabela2020_1.drop('Estado', axis=1)
tabela2020_1 = tabela2020_1.drop('Grupo vulnerável', axis=1)

tabela2020_1['Município'] = tabela2020_1['Município'].where(tabela2020_1['Município'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")


In [ ]:
# Salva a tabela unificada
tabela2020_1.to_excel("../Etapa 3/2020-1.xlsx", index=False)

## 4) Tratamento de 2020-2 a 2023-2 

In [2]:
# Le os arquivos de 2020-2 a 2023-2
tabela2020_2 = pd.read_excel("2020-2.xlsx", dtype=str)
tabela2021_1 = pd.read_excel("2021-1.xlsx", dtype=str)
tabela2021_2 = pd.read_excel("2021-2.xlsx", dtype=str)
tabela2022_1 = pd.read_excel("2022-1.xlsx", dtype=str)
tabela2022_2 = pd.read_excel("2022-2.xlsx", dtype=str)
tabela2023_1 = pd.read_excel("2023-1.xlsx", dtype=str)
tabela2023_2 = pd.read_excel("2023-2.xlsx", dtype=str)

In [3]:
# Lê arquivo com a lista de municípios do Brasil
arquivoComMunicipios = pd.read_csv("municipios.csv", dtype=str, delimiter=';', encoding = "ISO-8859-1")
arquivoComMunicipios['MUNICÍPIO - TOM'] = arquivoComMunicipios['MUNICÍPIO - TOM'].apply(unidecode)

# Lê arquivo com as categorias de profissão do ENEM
arquivoComProfissoes = pd.read_excel("profissoes.xlsx", dtype=str)
arquivoComProfissoes = arquivoComProfissoes.applymap(lambda x: x.upper() if isinstance(x, str) else x)

C:\Users\gabri\AppData\Local\Temp\ipykernel_9540\1232524672.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  arquivoComProfissoes = arquivoComProfissoes.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [4]:
# Unifica as tabelas
tabela2020_2_2023_2 = pd.concat([tabela2020_2, tabela2021_1, tabela2021_2, tabela2022_1, tabela2022_2, tabela2023_1, tabela2023_2], ignore_index=True)

In [ ]:
# Limpezas especificas
for coluna in tabela2020_2_2023_2.columns: tabela2020_2_2023_2[coluna] = tabela2020_2_2023_2[coluna].replace(['NAO SABE INFORMAR', 'N/D', "ATENDIMENTO INTERROMPIDO", "DENUNCIANTE NAO SOUBE INFORMAR", "NAO INFORMADO"], '')
tabela2020_2_2023_2 = tabela2020_2_2023_2.applymap(lambda x: x.strip() if isinstance(x, str) else x)

################################################################

tabela2020_2_2023_2['País'] = tabela2020_2_2023_2['País'].apply(PadronizaPais)

tabela2020_2_2023_2['UF'] = tabela2020_2_2023_2['UF'].str.strip()
tabela2020_2_2023_2['UF'] = tabela2020_2_2023_2['UF'].mask(~tabela2020_2_2023_2['UF'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

tabela2020_2_2023_2['Município'] = tabela2020_2_2023_2['Município'].apply(PadronizaMunicipio)
tabela2020_2_2023_2['Município'] = tabela2020_2_2023_2['Município'].where(tabela2020_2_2023_2['Município'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

################################################################

tabela2020_2_2023_2['Nacionalidade da vítima'] = tabela2020_2_2023_2['Nacionalidade da vítima'].apply(PadronizaPais)

tabela2020_2_2023_2['País da vítima'] = tabela2020_2_2023_2['País da vítima'].apply(PadronizaPais)

tabela2020_2_2023_2['UF da vítima'] = tabela2020_2_2023_2['UF da vítima'].str.strip()
tabela2020_2_2023_2['UF da vítima'] = tabela2020_2_2023_2['UF da vítima'].mask(~tabela2020_2_2023_2['UF da vítima'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

tabela2020_2_2023_2['Município da vítima'] = tabela2020_2_2023_2['Município da vítima'].apply(PadronizaMunicipio)
tabela2020_2_2023_2['Município da vítima'] = tabela2020_2_2023_2['Município da vítima'].where(tabela2020_2_2023_2['Município da vítima'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

tabela2020_2_2023_2['sl vitima naturalizado uf'] = tabela2020_2_2023_2['sl vitima naturalizado uf'].str.strip()
tabela2020_2_2023_2['sl vitima naturalizado uf'] = tabela2020_2_2023_2['sl vitima naturalizado uf'].mask(~tabela2020_2_2023_2['sl vitima naturalizado uf'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

tabela2020_2_2023_2['sl vitima naturalizado municipio'] = tabela2020_2_2023_2['sl vitima naturalizado municipio'].apply(PadronizaMunicipio)
tabela2020_2_2023_2['sl vitima naturalizado municipio'] = tabela2020_2_2023_2['sl vitima naturalizado municipio'].where(tabela2020_2_2023_2['sl vitima naturalizado municipio'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

################################################################

tabela2020_2_2023_2['Nacionalidade do suspeito'] = tabela2020_2_2023_2['Nacionalidade do suspeito'].apply(PadronizaPais)

tabela2020_2_2023_2['País do suspeito'] = tabela2020_2_2023_2['País do suspeito'].apply(PadronizaPais)

tabela2020_2_2023_2['UF do suspeito'] = tabela2020_2_2023_2['UF do suspeito'].str.strip()
tabela2020_2_2023_2['UF do suspeito'] = tabela2020_2_2023_2['UF do suspeito'].mask(~tabela2020_2_2023_2['UF do suspeito'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

tabela2020_2_2023_2['Município do suspeito'] = tabela2020_2_2023_2['Município do suspeito'].apply(PadronizaMunicipio)
tabela2020_2_2023_2['Município do suspeito'] = tabela2020_2_2023_2['Município do suspeito'].where(tabela2020_2_2023_2['Município do suspeito'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

tabela2020_2_2023_2['sl suspeito naturalizado uf'] = tabela2020_2_2023_2['sl suspeito naturalizado uf'].str.strip()
tabela2020_2_2023_2['sl suspeito naturalizado uf'] = tabela2020_2_2023_2['sl suspeito naturalizado uf'].mask(~tabela2020_2_2023_2['sl suspeito naturalizado uf'].isin(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]), '')

tabela2020_2_2023_2['sl suspeito naturalizado municipio'] = tabela2020_2_2023_2['sl suspeito naturalizado municipio'].apply(PadronizaMunicipio)
tabela2020_2_2023_2['sl suspeito naturalizado municipio'] = tabela2020_2_2023_2['sl suspeito naturalizado municipio'].where(tabela2020_2_2023_2['sl suspeito naturalizado municipio'].isin(arquivoComMunicipios['MUNICÍPIO - TOM']), "")

################################################################

tabela2020_2_2023_2['Sexo da vítima'] = tabela2020_2_2023_2['Sexo da vítima'].mask(~tabela2020_2_2023_2['Sexo da vítima'].isin(['FEMININO', 'MASCULINO']), '')
tabela2020_2_2023_2['Sexo do suspeito'] = tabela2020_2_2023_2['Sexo do suspeito'].mask(~tabela2020_2_2023_2['Sexo do suspeito'].isin(['FEMININO', 'MASCULINO']), '')

tabela2020_2_2023_2['Faixa etária da vítima'] = tabela2020_2_2023_2['Faixa etária da vítima'].replace(['10 ANOS', '11 ANOS'], '10 A 11 ANOS')
tabela2020_2_2023_2['Faixa etária da vítima'] = tabela2020_2_2023_2['Faixa etária da vítima'].replace(['12 ANOS', '13 ANOS', '14 ANOS'], '12 A 14 ANOS')
tabela2020_2_2023_2['Faixa etária do suspeito'] = tabela2020_2_2023_2['Faixa etária do suspeito'].replace(['15 ANOS', '16 ANOS', '17 ANOS'], '15 A 17 ANOS')
tabela2020_2_2023_2['Faixa etária do suspeito'] = tabela2020_2_2023_2['Faixa etária do suspeito'].replace(['12 ANOS', '13 ANOS', '14 ANOS'], '12 A 14 ANOS')

tabela2020_2_2023_2['Deficiência da vítima'] = tabela2020_2_2023_2['Deficiência da vítima'].replace(['NAO'], 'NAO TEM DEFICIENCIA')
tabela2020_2_2023_2['Deficiência do suspeito'] = tabela2020_2_2023_2['Deficiência do suspeito'].replace(['NAO'], 'NAO TEM DEFICIENCIA')

tabela2020_2_2023_2['Doença rara da vítima'] = tabela2020_2_2023_2['Doença rara do suspeito'].replace(['NAO'], 'NAO TEM DOENCA RARA')
tabela2020_2_2023_2['Doença rara do suspeito'] = tabela2020_2_2023_2['Doença rara do suspeito'].replace(['NAO'], 'NAO TEM DOENCA RARA')

tabela2020_2_2023_2['sl vitima cadastro'] = tabela2020_2_2023_2['sl vitima cadastro'].replace(['COMUNIDADE', 'FAMILIA'], 'COMUNIDADE/FAMILIA')
tabela2020_2_2023_2['Natureza Jurídica do Suspeito'] = tabela2020_2_2023_2['Natureza Jurídica do Suspeito'].replace(['COMUNIDADE', 'FAMILIA'], 'COMUNIDADE/FAMILIA')

tabela2020_2_2023_2['Orientação sexual da vítima'] = tabela2020_2_2023_2['Orientação sexual da vítima'].replace(['HOMOSSEXUAL.LESBICA', 'HOMOSSEXUAL.GAY'], 'HOMOSSEXUAL')
tabela2020_2_2023_2['Orientação sexual do suspeito'] = tabela2020_2_2023_2['Orientação sexual do suspeito'].replace(['HOMOSSEXUAL.LESBICA', 'HOMOSSEXUAL.GAY'], 'HOMOSSEXUAL')

tabela2020_2_2023_2['Grau de instrução da vítima'] = tabela2020_2_2023_2['Grau de instrução da vítima'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')
tabela2020_2_2023_2['Grau de instrução do suspeito'] = tabela2020_2_2023_2['Grau de instrução do suspeito'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')

tabela2020_2_2023_2['Religião da vítima'] = tabela2020_2_2023_2['Religião da vítima'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')
tabela2020_2_2023_2['Religião do suspeito'] = tabela2020_2_2023_2['Religião do suspeito'].replace(['ANALFABETO/SEM INSTRUCAO'], 'ANALFABETO')

substituicoes = {
    'PRETO': '',
    'PARDO': '',
    'PARDA': '',
    'COMUNIDADES QUILOMBOLAS': 'QUILOMBOLA',
    'POVOS INDIGENAS': 'INDIGENA',
    'DEMAIS COMUNIDADES TRADICIONAIS': 'POVOS E COMUNIDADES TRADICIONAIS',
    'POVOS CIGANOS': 'CIGANO'
}
tabela2020_2_2023_2['Etnia da vítima'] = tabela2020_2_2023_2['Etnia da vítima'].replace(substituicoes)
tabela2020_2_2023_2['Etnia do Suspeito'] = tabela2020_2_2023_2['Etnia do Suspeito'].replace(substituicoes)

substituicoes = {
    'ALUNO (A)': 'ALUNO(A)',
    'AMIGO (A)': 'AMIGO(A)',
    'COMPANHEIRO (A)': 'COMPANHEIRO(A)',
    'CUIDADOR (A)': 'CUIDADOR(A)',
    'CUNHADO (A)': 'CUNHADO(A)',
    'EMPREGADO(HIERARQUICAMENTE INFERIOR)': 'EMPREGADO (HIERARQUICAMENTE INFERIOR)',
    'EMPREGADOR/ PATRAO(HIERARQUICAMENTE SUPERIOR)': 'EMPREGADOR/PATRAO (HIERARQUICAMENTE SUPERIOR)',
    'EMPREGADOR/PATRAO(HIERARQUICAMENTE SUPERIOR)': 'EMPREGADOR/PATRAO (HIERARQUICAMENTE SUPERIOR)',
    'ESPOSA': 'ESPOSO(A)',
    'MARIDO': 'ESPOSO(A)',
    'EX-COMPANHEIRO (A)': 'EX-COMPANHEIRO(A)',
    'EX-ESPOSA': 'EX-ESPOSO(A)',
    'EX-MARIDO': 'EX-ESPOSO(A)',
    'EX-NAMORADO (A)': 'EX-NAMORADO(A)',
    'FILHO (A)': 'FILHO(A)',
    'IRMAO (A)': 'IRMAO(A)',
    'LIDER RELIGIOSO': 'LIDER RELIGIOSO(A)',
    'MORA NA MESMA RESIDENCIA MAS NAO E FAMILIAR': 'MORA/MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR',
    'MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR': 'MORA/MOROU NA MESMA RESIDENCIA MAS NAO E FAMILIAR',
    'FAMILIARES': 'OUTROS',
    'OUTROS FAMILIARES': 'OUTROS',
    'OUTROS PROFISSIONAIS DA EDUCACAO': 'OUTROS',
    'PRESTADOR DE SERVICO': 'PRESTADOR(A) DE SERVICO',
    'TIO (A)': 'TIO(A)',
    'TREINADOR/TECNICO': 'TREINADOR(A)/TECNICO(A)',
    'TRISAVO(O)': 'TRISAVO(O)',
    'VIZINHO (A)': 'VIZINHO(A)'
}
tabela2020_2_2023_2['Relação vítima-suspeito'] = tabela2020_2_2023_2['Relação vítima-suspeito'].replace(substituicoes)

substituicoes = {
    '1 A 3 SM': '1 A 3 SM',
    '3 A 5 SM': '3 A 5 SM',
    '5 A 15 SM': '5 A 15 SM',
    '6 A 15 SM': '5 A 15 SM',
    '8 A 15 SM': '5 A 15 SM',
    'ACIMA DE 15 SM': 'ACIMA DE 15 SM',
    'ATE 1 SM': 'ATE 1 SM',
    'ATE 1/2 SALARIO MINIMO': 'ATE 1/2 SM',
    'MAIS DE 1 A 2 SALARIOS MINIMOS': '1 A 3 SM',
    'MAIS DE 1/2 A 1 SALARIO MINIMO': 'ATE 1 SM',
    'MAIS DE 10 A 20 SALARIOS MINIMOS': '10 A 20 SM',
    'MAIS DE 2 A 5 SALARIOS MINIMOS': '3 A 5 SM',
    'MAIS DE 20 SALARIOS MINIMOS': 'MAIS DE 20 SM',
    'MAIS DE 5 A 10 SALARIOS MINIMOS': '5 A 15 SM',
    'SEM RENDIMENTO': 'SEM RENDIMENTO'
}
tabela2020_2_2023_2['Faixa de renda da vítima'] = tabela2020_2_2023_2['Faixa de renda da vítima'].replace(substituicoes)

tabela2020_2_2023_2['Violações'] = tabela2020_2_2023_2['Violações'].str.replace(';', '', regex=False)

tabela2020_2_2023_2['Cenário da violação'] = tabela2020_2_2023_2['Cenário da violação'].replace(['AMBIENTE DE LAZER/ESPORTE/ENTRETENIMENTO'], 'AMBIENTE DE LAZER')
tabela2020_2_2023_2['Cenário da violação'] = tabela2020_2_2023_2['Cenário da violação'].replace(['TAXI/TRANSPORTE APLICATIVO'], 'TAXI')


tabela2020_2_2023_2 = tabela2020_2_2023_2.drop('Hash', axis=1)
tabela2020_2_2023_2 = tabela2020_2_2023_2.drop('Grupo vulnerável', axis=1)

# Padroniza conforme dicionario de dados do ENEM
tabela2020_2_2023_2['Profissão da vítima'] = tabela2020_2_2023_2['Profissão da vítima'].apply(lambda profissao: ClassificaProfissao(profissao, arquivoComProfissoes))
tabela2020_2_2023_2['Profissão do suspeito'] = tabela2020_2_2023_2['Profissão do suspeito'].apply(lambda profissao: ClassificaProfissao(profissao, arquivoComProfissoes))

In [ ]:
# Salva a tabela unificada
tabela2020_2_2023_2.to_excel("../Etapa 3/2020-2 a 2023-2.xlsx", index=False)